# Subclassing `ASFProduct`

`ASFProduct` is the base class for all search result objects as of asf-search v7.0.0. There are several subclasses of `ASFProduct` that asf-search uses for specific platforms and product types with unique properties/functionality.

In [ ]:
import asf_search as asf
products = ['S1A_IW_SLC__1SDV_20231226T162948_20231226T163016_051828_0642C6_272F-SLC', 'S1_185682_IW2_20210224T161634_VV_035E-BURST','S1-GUNW-D-R-087-tops-20190301_20190223-161540-20645N_18637N-PP-7a85-v2_0_1-unwrappedPhase','ALPSRP111041130-RTC_HI_RES', 'UA_newyor_03204_22005-013_22010-002_0014d_s01_L090_01-INTERFEROMETRY']
results = asf.product_search(product_list=products)
results

Notice the different type in the `results` list: `S1Product`, `S1BURSTProduct`, `ARIAS1GUNWProduct`, `ALOSProduct`, and `UAVSARProduct`.
Each of these classes are subclassed from `ASFProduct` in some way.

Let's compare the `properties` of `S1Product` and `ALOSProduct`

In [ ]:
s1, uavsar, s1Burst, ariaGunw, alos, = results

def compare_properties(lhs: asf.ASFProduct, rhs: asf.ASFProduct):
    # Compares properties of two ASFProduct objects in a color coded table
    # values printed in red are missing from that product type altogether
    
    # Color Coding
    RED = '\033[31m'
    GREEN = '\033[32m'
    BLUE  = '\033[34m'
    RESET = '\033[0m'

    print(f'\t{GREEN}{type(lhs)}{RESET}\t{BLUE}{type(rhs)}{RESET}')
    
    keys = {*lhs.properties.keys(), *rhs.properties.keys()}
    for key in keys:
        print(f"{key}:\n\t{GREEN}{lhs.properties.get(key, f'{RED}None')}{RESET}\t{BLUE}{rhs.properties.get(key, f'{RED}None')}{RESET}\n")

compare_properties(s1, uavsar)

Notice a few properties (marked in red) are missing from each product properties dict. For example, `S1Product` has `pgeVersion`, while `UAVSARProduct` has `insarStackId`. 

Moreover, `S1Product` has one major difference with `UAVSARProduct`: `S1Product` inherits from `ASFStackableProduct` (see section below).

In [ ]:
print(f"{s1.properties['fileID']}\n\t{s1.baseline}\n")
print(f"{uavsar.properties['fileID']}\n\t{uavsar.baseline}")

# `ASFStackableProduct`

`ASFStackableProduct` is an important `ASFProduct` subclass, from which all stackable products meant for time-series analysis are derived from. `ASFStackableProduct` has a class enum, `BaselineCalcType` that determines how asf-search will handle perpendicular stack calculations. Each subclass keeps track of their baseline calculation type via the `baseline_type` property.

Inherits: `ASFProduct`

Inherited By:
- `ALOSProduct`
- `ERSProduct`
- `JERSProduct`
- `RadarsatProduct`
- `S1Product`
    - `S1BurstProduct`
    - `OPERAS1Product` (Stacking currently disabled)
    - `ARIAS1GUNWProduct` (Stacking currently disabled)

Key Methods:
- `get_baseline_calc_properties()`
- `get_stack_opts()` (Overrides `ASFproduct`)
- `is_valid_reference()`
- `get_default_baseline_product_type()`

Key Definitions:
class enum `BaselineCalcType`:
- `PRE_CALCULATED` Has pre-calculated `insarBaseline` value that will be used for perpendicular calculations
- `CALCULATED` Uses position/velocity state vectors and ascending node time for perpendicular calculations

Key Fields:
- `baseline`
- `baseline_type` (`BaselineCalcType.PRE_CALCULATED` by default or `BaselineCalcType.CALCULATED`)



In [ ]:
print(f"Baseline Calculation Types")
print(f"ASFProduct:\t {asf.ASFStackableProduct.baseline_type}")
print(f"ALOSProduct:\t {alos.baseline_type}")
print(f"S1Product:\t {s1.baseline_type}")

`ASFStackableProduct` subclasses even have their own stack search option methods. The `ASFStackableProduct` implementation of `get_stack_opts()` returns the commonly used params for pre-calculated datasets (processing level and insar stack ID), but subclasses like `S1Product` and `S1BurstProduct` use their own approach. 

In [ ]:
print(f"S1Product:\n{s1.get_stack_opts()}\n")
print(f"S1BURSTProduct:\n{s1Burst.get_stack_opts()}\n")
print(f"ALOSProduct:\n{alos.get_stack_opts()}")

# Writing Custom `ASFProduct` Subclasses
Because `ASFProduct` is built for subclassing, that means users can provide their own custom subclasses dervied directly from `ASFProduct` or even from a pre-existing subclass like `S1Product` or `OperaS1Product`.

In this example we subclass `S1Product`, and overrides the default `ASFProduct.stack()` with one that returns a _list_ of `S1BurstProduct` stacks based on an area of interest, modify `geojson()` to return state vectors, and add a new helper method for getting raw umm CMR response!

In [ ]:
from typing import Union
from asf_search import ASFSearchOptions, ASFSession
from asf_search.ASFSearchOptions import ASFSearchOptions
from typing import Dict

class MyCustomS1Subclass(asf.S1Product):
    def __init__(
                #default ASFProduct arguments
                self, args: Dict = {}, session: ASFSession = ASFSession(), 
                #custom properties
                custom_properties: Dict = {}
            ):
        super().__init__(args, session)

        # unique property of MyCustomClass
        self.custom_properties = custom_properties
    
    # write custom methods
    def as_umm_json(self) -> Dict:
        return { 'umm': self.umm, 'meta': self.meta }
    
    # Override built in ASFProduct methods, like `geojson()`, `get_stack_opts()`, or `get_default_baseline_product_type()`
    def geojson(self) -> Dict:
        output = {
            **super().geojson()
        }

        output['properties'] = {
            **output['properties'],
            'customProperties': self.custom_properties
        }

        return output
    
    def get_stack_opts(self, opts: ASFSearchOptions = None) -> ASFSearchOptions:
        # use S1Product's stack opt already written functionality
        opts = super().get_stack_opts(opts)

        # but use some new product type to build the stack instead of S1Product's default, "SLC"
        opts.processingLevel = self.get_default_baseline_product_type()

        return opts
    
    @staticmethod
    def get_default_baseline_product_type() -> Union[str, None]:
        """
        Returns the product type to search for when building a baseline stack.
        """
        return 'NEW_PRODUCT_TYPE'

customS1SubclassProduct = MyCustomS1Subclass({'umm': s1.umm, 'meta': s1.meta}, session=s1.session, custom_properties={'customProperty': 'This is a special property'})

customS1SubclassProduct.geojson()

Notice the `customProperties` field in the output from `geojson()`.
`get_stack_opts()` should also use S1Product's stack ops, but with `NEW_PRODUCT_TYPE` instead of `SLC`:

In [ ]:
print(customS1SubclassProduct.get_stack_opts())